# 환경설정

In [6]:
!nvidia-smi

Wed May 22 06:00:19 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
%cd /content/drive/MyDrive/datasets/

/content/drive/MyDrive/datasets


In [4]:
!ls

food10.zip  single_prediction  test_set  training_set


In [5]:
!unzip food10.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: bibimbap/train_086851.jpg  
  inflating: bibimbap/train_086852.jpg  
  inflating: bibimbap/train_086853.jpg  
  inflating: bibimbap/train_086854.jpg  
  inflating: bibimbap/train_086855.jpg  
  inflating: bibimbap/train_086856.jpg  
  inflating: bibimbap/train_086857.jpg  
  inflating: bibimbap/train_086858.jpg  
  inflating: bibimbap/train_086859.jpg  
  inflating: bibimbap/train_086860.jpg  
  inflating: bibimbap/train_086861.jpg  
  inflating: bibimbap/train_086862.jpg  
  inflating: bibimbap/train_086863.jpg  
  inflating: bibimbap/train_086864.jpg  
  inflating: bibimbap/train_086865.jpg  
  inflating: bibimbap/train_086866.jpg  
  inflating: bibimbap/train_086867.jpg  
  inflating: bibimbap/train_086868.jpg  
  inflating: bibimbap/train_086869.jpg  
  inflating: bibimbap/train_086870.jpg  
  inflating: bibimbap/train_086871.jpg  
  inflating: bibimbap/train_086872.jpg  
  inflating: bibimbap/train_086873.jpg  
  inflating: bibimbap

# 라이브러리 불러오기(데이터 처리)



In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam

In [8]:
# 학습데이터(Train) 경로
data_dir = '/content/drive/MyDrive/datasets/food10/'
data = tf.keras.preprocessing.image_dataset_from_directory(data_dir)

Found 5113 files belonging to 10 classes.


In [14]:
data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [9]:
# 데이터 인공지능이 학습 가능한 형태로 변환
datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [10]:
width = 228
height = 228
channels = 3
batch_size = 32
image_shape = (height, width, channels)
image_size = (height, width)

In [11]:
# 데이터 변환
train_data = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

Found 4094 images belonging to 10 classes.


In [12]:
train_data.class_indices

{'bibimbap': 0,
 'chiffon_cake': 1,
 'donut': 2,
 'egg_roll': 3,
 'garlic_bread': 4,
 'gyoza': 5,
 'kabob': 6,
 'omelette': 7,
 'pizza': 8,
 'sashimi': 9}

In [13]:
val_data = datagen.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

Found 1019 images belonging to 10 classes.


In [15]:
# 클래스 명칭설정
num_classes = len(data.class_names)

In [16]:
num_classes

10

In [17]:
data.class_names

['bibimbap',
 'chiffon_cake',
 'donut',
 'egg_roll',
 'garlic_bread',
 'gyoza',
 'kabob',
 'omelette',
 'pizza',
 'sashimi']

In [22]:
# 학습할 이미지에 대한 Preview
def show_image(data):
  plt.figure(figsize=(15,15))
  for images, labels in data.take(1):
    for i in range(9):
      ax = plt.subplot(3,3,i + 1)
      ax.imshow(images[i].numpy().astype("uint8"))
      ax.axis("off")

show_image(data)

Output hidden; open in https://colab.research.google.com to view.

# 인공지능 알고리즘 설계